In [1]:
# Data Manipulation and Preparation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Evaluation metrics and visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Ensure GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

import itertools
import time
import csv
import json
from IPython.display import Audio, display
import winsound
from threading import Thread

Using device: cuda


In [2]:
# Load cleaned dataset
df = pd.read_csv('BTC_1h_w_Diffs.csv', parse_dates=['open_time'])

# Convert to datetime 
df['open_time'] = pd.to_datetime(df['open_time'])
df['close_time'] = pd.to_datetime(df['close_time'])

In [1]:
def create_sequences(data, target, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        seq = data[i:i + window_size]
        label = target[i + window_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

def prepare_data(df, target_column, window_size, feature_columns):
    X = df[feature_columns].values
    y = df[target_column].values.reshape(-1, 1)
    
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

    return X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y

class LSTMModel(nn.Module):
    def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, num_layers=1, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc1 = nn.Linear(lstm_dim, dense_dim)
        self.activation = nn.Tanh()
        self.fc2 = nn.Linear(dense_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

def train_and_evaluate(df, target_column, feature_columns, params, file_prefix='results', use_early_stopping=True, plot_loss=False):
    result = {}
    try:
        # Extract parameters
        window_size = params['window_size']
        lstm_dim = params['lstm_dim']
        dense_dim = params['dense_dim']
        num_layers = params['num_layers']
        dropout = params['dropout']
        lr = params['lr']
        batch_size = params['batch_size']
        num_epochs = params['num_epochs']
        optimizer_type = params['optimizer_type']
        patience = params['patience']

        # Clear CUDA cache
        torch.cuda.empty_cache()

        # Prepare data
        X_train_seq, X_test_seq, y_train_seq, y_test_seq, scaler_y = prepare_data(df, target_column, window_size, feature_columns)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
        X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
        y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

        # Model parameters
        input_dim = X_train_tensor.shape[2]
        output_dim = 1

        model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout).to(device)

        # Define loss and optimizer
        criterion = nn.MSELoss()
        optimizer = getattr(optim, optimizer_type)(model.parameters(), lr=lr)

        # Training loop
        best_loss = float('inf')
        patience_counter = 0
        start_time = time.time()

        training_losses = []
        validation_losses = []

        for epoch in range(num_epochs):
            model.train()
            epoch_train_loss = 0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                epoch_train_loss += loss.item()

            epoch_train_loss /= len(train_loader)
            training_losses.append(epoch_train_loss)

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    val_loss += criterion(outputs, labels).item()

            val_loss /= len(test_loader)
            validation_losses.append(val_loss)

            # if use_early_stopping:
            #     if val_loss < best_loss:
            #         best_loss = val_loss
            #         patience_counter = 0
            #         torch.save(model.state_dict(), f'{file_prefix}_{target_column}_best_model.pth')
            #     else:
            #         patience_counter += 1
            #         if patience_counter >= patience:
            #             break

        training_time = time.time() - start_time

        # # Save the final model if early stopping was not used or not triggered
        # if not use_early_stopping or patience_counter < patience:
        #     torch.save(model.state_dict(), f'{file_prefix}_{target_column}_final_model.pth')

        # Evaluate the model
        model.eval()
        evaluation_start_time = time.time()
        with torch.no_grad():
            train_predictions = model(X_train_tensor).cpu().numpy()
            test_predictions = model(X_test_tensor).cpu().numpy()

            # Inverse transform predictions and actual values
            y_train_seq = y_train_seq.reshape(-1, 1)
            y_test_seq = y_test_seq.reshape(-1, 1)
            train_predictions_inverse = scaler_y.inverse_transform(train_predictions)
            test_predictions_inverse = scaler_y.inverse_transform(test_predictions)
            y_train_inverse = scaler_y.inverse_transform(y_train_seq)
            y_test_inverse = scaler_y.inverse_transform(y_test_seq)

            # # Invert log transformation
            # train_predictions_inverse = np.exp(train_predictions_inverse) - 1
            # test_predictions_inverse = np.exp(test_predictions_inverse) - 1
            # y_train_inverse = np.exp(y_train_inverse) - 1
            # y_test_inverse = np.exp(y_test_inverse) - 1

            train_mse = mean_squared_error(y_train_inverse, train_predictions_inverse)
            test_mse = mean_squared_error(y_test_inverse, test_predictions_inverse)
            train_mae = mean_absolute_error(y_train_inverse, train_predictions_inverse)
            test_mae = mean_absolute_error(y_test_inverse, test_predictions_inverse)
            train_rmse = np.sqrt(train_mse)
            test_rmse = np.sqrt(test_mse)
            train_r2 = r2_score(y_train_inverse, train_predictions_inverse)
            test_r2 = r2_score(y_test_inverse, test_predictions_inverse)

            # Avoid division by zero by adding a small epsilon value to the denominator
            train_mape = np.mean(np.abs((y_train_inverse - train_predictions_inverse) / (y_train_inverse + epsilon))) * 100
            test_mape = np.mean(np.abs((y_test_inverse - test_predictions_inverse) / (y_test_inverse + epsilon))) * 100

            train_directional_acc = np.mean(np.sign(y_train_inverse[1:] - y_train_inverse[:-1]) == np.sign(train_predictions_inverse[1:] - train_predictions_inverse[:-1]))
            test_directional_acc = np.mean(np.sign(y_test_inverse[1:] - y_test_inverse[:-1]) == np.sign(test_predictions_inverse[1:] - test_predictions_inverse[:-1]))

        evaluation_time = time.time() - evaluation_start_time

        result = {
            "window_size": window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "train_mse": train_mse,
            "test_mse": test_mse,
            "train_mae": train_mae,
            "test_mae": test_mae,
            "train_rmse": train_rmse,
            "test_rmse": test_rmse,
            "train_r2": train_r2,
            "test_r2": test_r2,
            "train_mape": train_mape,
            "test_mape": test_mape,
            "train_directional_acc": train_directional_acc,
            "test_directional_acc": test_directional_acc,
            "training_time": training_time,
            "evaluation_time": evaluation_time,
            "patience": patience
        }

        # Save result to a CSV file
        with open(f'{file_prefix}_{target_column}.csv', 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=result.keys())
            if f.tell() == 0:
                writer.writeheader()  # Write header only if file is empty
            writer.writerow(result)

        # Print summary of the results
        print(f"Results: Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
        print(f"Train MAPE: {train_mape:.4f}, Test MAPE: {test_mape:.4f}")
        print(f"Train Directional Accuracy: {train_directional_acc:.4f}, Test Directional Accuracy: {test_directional_acc:.4f}")
        print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

        # Plot loss over epochs if specified
        if plot_loss:
            plot_loss_over_epochs(training_losses, validation_losses, f'{file_prefix}_{target_column}', start_epoch=3)

    except Exception as e:
        print(f"An error occurred: {e}")

        result = {
            "window_size": window_size,
            "lstm_dim": lstm_dim,
            "num_layers": num_layers,
            "dense_dim": dense_dim,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch_size,
            "num_epochs": num_epochs,
            "optimizer_type": optimizer_type,
            "error": str(e)
        }

    return result

NameError: name 'nn' is not defined

In [14]:
# Define parameter space
parameter_space = {
    'window_size': [24],
    'lstm_dim': [96, 128, 150],
    'dense_dim': [96, 128, 150],
    'num_layers': [1],
    'dropout': [0.001, 0.0001],
    'lr': [0.0001, 0.001],
    'num_epochs': [50, 150, 250],
    'batch_size': [64],
    'optimizer_type': ['Adam'],
    'patience': [24],
    'activation_function': [nn.Tanh]
}

In [17]:
# Define target column
target_column = ['diff']

# Define feature combination
features = ['diff_high', 'diff_low', 'diff', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote', 'high_p', 'low_p', 'close_p']

In [20]:
## THIS RUN HAS MODIFIED MAPE CODE BECAUSE OF INFINITIES ##
epsilon = 1e-10

# Start total timer
start = time.time()

# Calculate total number of models
total_models = len(list(itertools.product(*parameter_space.values())))

# Initialize model counter
model_counter = 1

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model {model_counter}/{total_models} with parameters: {param_dict}\n")
    
    # Run with fixed epochs
    print('Results:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results1.0', use_early_stopping=False, plot_loss=False)
    
    # Increment model counter
    model_counter += 1

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")


Running model 1/108 with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results:

Results: Train MAE: 140.7869, Test MAE: 138.1188
Train MAPE: 929269801.2990, Test MAPE: 1724366452.7471
Train Directional Accuracy: 0.4567, Test Directional Accuracy: 0.4502
Training Time: 55.2459 seconds, Evaluation Time: 16.6552 seconds

Running model 2/108 with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 150, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results:

Results: Train MAE: 139.9953, Test MAE: 137.2090
Train MAPE: 3718935585.8137, Test MAPE: 95755010.9785
Train Directional Accuracy: 0.5024, Test Directional Accuracy: 0.4

In [21]:
# Define feature combination
features = ['high', 'low', 'close', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote', 'high_p', 'low_p', 'close_p']

In [22]:
# Start total timer
start = time.time()

# Calculate total number of models
total_models = len(list(itertools.product(*parameter_space.values())))

# Initialize model counter
model_counter = 1

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model {model_counter}/{total_models} with parameters: {param_dict}\n")
    
    # Run with fixed epochs
    print('Results:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results1.1', use_early_stopping=False, plot_loss=True)
    
    # Increment model counter
    model_counter += 1

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")

Running model 1/108 with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results:

Results: Train MAE: 140.6731, Test MAE: 137.7948
Train MAPE: 1482427273.3644, Test MAPE: 2048779639.0371
Train Directional Accuracy: 0.4847, Test Directional Accuracy: 0.4802
Training Time: 49.1068 seconds, Evaluation Time: 0.2745 seconds

An error occurred: name 'plot_loss_over_epochs' is not defined
Running model 2/108 with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 150, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results:

Results: Train MAE: 142.9539, Test MAE: 139.9863
Train MAPE: 6215235438.5713, Test MAPE: 3435324908.6029
T

In [26]:
torch.cuda.empty_cache()

In [23]:
# Define feature combination
features = ['diff_high', 'diff_low', 'diff', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote']

In [27]:
# Start total timer
start = time.time()

# Calculate total number of models
total_models = len(list(itertools.product(*parameter_space.values())))

# Initialize model counter
model_counter = 1

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model {model_counter}/{total_models} with parameters: {param_dict}\n")
    
    # Run with fixed epochs
    print('Results:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results1.2', use_early_stopping=False, plot_loss=False)
    
    # Increment model counter
    model_counter += 1

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")

Running model 1/108 with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 50, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results:

Results: Train MAE: 140.6441, Test MAE: 137.2365
Train MAPE: 3270485218.9396, Test MAPE: 1206302523.7624
Train Directional Accuracy: 0.5086, Test Directional Accuracy: 0.4964
Training Time: 54.2602 seconds, Evaluation Time: 0.1342 seconds

Running model 2/108 with parameters: {'window_size': 24, 'lstm_dim': 96, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.001, 'lr': 0.0001, 'num_epochs': 150, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 24, 'activation_function': <class 'torch.nn.modules.activation.Tanh'>}

Results:

Results: Train MAE: 149.2621, Test MAE: 143.2439
Train MAPE: 10211963600.2057, Test MAPE: 4626634311.8722
Train Directional Accuracy: 0.5014, Test Directional Accuracy: 

In [ ]:
# Define feature combination
features = ['diff_high', 'diff_low', 'high', 'low', 'close', 'diff', 'volume',
       'quote_vol', 'count', 'buy_base', 'buy_quote']

In [ ]:
# Start total timer
start = time.time()

# Calculate total number of models
total_models = len(list(itertools.product(*parameter_space.values())))

# Initialize model counter
model_counter = 1

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model {model_counter}/{total_models} with parameters: {param_dict}\n")
    
    # Run with fixed epochs
    print('Results:\n')
    train_and_evaluate(df, target_column=target_column, feature_columns=features, params=param_dict, file_prefix='results1.3', use_early_stopping=False, plot_loss=False)
    
    # Increment model counter
    model_counter += 1

# Calculate total time
end = time.time()
total_time = end - start

# Print the total time taken
print(f"Total time taken: {total_time} seconds")